In [3]:
%%capture
!pip install "numpy>=1.12.1"
!pip install "scipy>=1.0.1"
!pip install "torch>=2.0.0"
!pip install accelerate==0.23.0
# !pip install bitsandbytes==0.41.0
!pip install bitsandbytes
!pip install peft==0.5.0
!pip install transformers==4.34.0
!pip install pillow==10.0.1
!pip install llama-cpp-python
!pip install datasets
!pip install zstandard
!pip install jsonlines
!pip install sentencepiece
!pip install fire
!pip install datasketch==1.5.9
!pip install nltk==3.8.1
!pip install scikit-learn==1.3.0
!pip install pytest

In [ ]:
import os
os._exit(00) # Перезапустить ядро

In [1]:

%mkdir /tmp
%ls

ЋиЁЎЄ  ў бЁ­в ЄбЁбҐ Є®¬ ­¤л.


 ’®¬ ў гбва®©бвўҐ E ­Ґ Ё¬ҐҐв ¬ҐвЄЁ.
 ‘ҐаЁ©­л© ­®¬Ґа в®¬ : C884-71E7

 ‘®¤Ґа¦Ё¬®Ґ Ї ЇЄЁ E:\PyCharm Community Edition 2021.1.1\projects\LLM_LoRa

03.07.2024  23:07    <DIR>          .
03.07.2024  23:07    <DIR>          ..
03.07.2024  17:27    <DIR>          .idea
03.07.2024  12:42            22я395 extraction-saiga.ipynb
03.07.2024  23:07           277я848 fine-tune-llama.ipynb
03.07.2024  14:59    <DIR>          venv
               2 д ©«®ў        300я243 Ў ©в
               4 Ї Ї®Є  342я691я061я760 Ў ©в бў®Ў®¤­®


In [2]:
!git clone https://github.com/ggerganov/llama.cpp.git
!git clone https://github.com/IlyaGusev/rulm.git

Cloning into 'llama.cpp'...
Cloning into 'rulm'...
Updating files:  33% (118/354)
Updating files:  34% (121/354)
Updating files:  35% (124/354)
Updating files:  36% (128/354)
Updating files:  37% (131/354)
Updating files:  38% (135/354)
Updating files:  39% (139/354)
Updating files:  40% (142/354)
Updating files:  41% (146/354)
Updating files:  42% (149/354)
Updating files:  43% (153/354)
Updating files:  44% (156/354)
Updating files:  45% (160/354)
Updating files:  46% (163/354)
Updating files:  47% (167/354)
Updating files:  48% (170/354)
Updating files:  49% (174/354)
Updating files:  50% (177/354)
Updating files:  51% (181/354)
Updating files:  52% (185/354)
Updating files:  53% (188/354)
Updating files:  54% (192/354)
Updating files:  55% (195/354)
Updating files:  56% (199/354)
Updating files:  57% (202/354)
Updating files:  58% (206/354)
Updating files:  59% (209/354)
Updating files:  60% (213/354)
Updating files:  61% (216/354)
Updating files:  62% (220/354)
Updating files:  63

In [2]:
import torch
import pandas as pd
from peft import PeftModel, PeftConfig
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForTokenClassification, AutoConfig, GenerationConfig
from transformers import Trainer, TrainingArguments, logging, TrainerCallback, TrainerState, TrainerControl, BitsAndBytesConfig
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR
from peft import get_peft_model, LoraConfig, prepare_model_for_kbit_training
import torch.nn.functional as F
from datasets import load_dataset
import time
from typing import Any, List, Mapping, Optional
import transformers
import os
from pathlib import Path

RuntimeError: Failed to import transformers.trainer because of the following error (look up to see its traceback):
cannot import name 'insecure_hashlib' from 'huggingface_hub.utils' (E:\PyCharm Community Edition 2021.1.1\projects\LLM_LoRa\venv\Lib\site-packages\huggingface_hub\utils\__init__.py)

In [5]:
MODEL_NAME = "IlyaGusev/saiga2_7b_lora"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

st_time = time.time()

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype= torch.float16,
#     bnb_4bit_use_double_quant=False,
# )


config = PeftConfig.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    load_in_8bit = True,
    torch_dtype=torch.float16,
    device_map="auto"
)
model = PeftModel.from_pretrained(
    model,
    MODEL_NAME,
    torch_dtype=torch.float16,
    is_trainable = True
).to(device)

model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
print(generation_config)
print(f'Прошло времени {time.time() - st_time}')

adapter_config.json:   0%|          | 0.00/476 [00:00<?, ?B/s]

E:\PyCharm Community Edition 2021.1.1\projects\LLM_LoRa\venv\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--IlyaGusev--saiga2_7b_lora. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
E:\PyCharm Community Edition 2021.1.1\projects\LLM_LoRa\venv\Lib\site-packages\h

config.json:   0%|          | 0.00/554 [00:00<?, ?B/s]

E:\PyCharm Community Edition 2021.1.1\projects\LLM_LoRa\venv\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--TheBloke--Llama-2-7B-fp16. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


ImportError: Using `load_in_8bit=True` requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes `pip install -i https://test.pypi.org/simple/ bitsandbytes` or pip install bitsandbytes` 

In [5]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}")

In [6]:
config

LoraConfig(peft_type='LORA', auto_mapping=None, base_model_name_or_path='TheBloke/Llama-2-7B-fp16', revision=None, task_type='CAUSAL_LM', inference_mode=True, r=16, target_modules=['q_proj', 'v_proj', 'k_proj', 'o_proj'], lora_alpha=16, lora_dropout=0.05, fan_in_fan_out=False, bias='none', modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None)

In [7]:
print_trainable_parameters(model)

trainable params: 16777216 || all params: 6755192832 || trainable%: 0.24836028248556738


### Загружаем датасет

In [8]:
from datasets import load_dataset

In [9]:
data = load_dataset(
    "json", 
    data_files={'train' : '/kaggle/input/fine-tunning-llama/train.json' , 'validation' : '/kaggle/input/fine-tunning-llama/val.json'}
)

data

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-b9460e3075a3c6fd/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['system', 'user', 'bot'],
        num_rows: 22
    })
    validation: Dataset({
        features: ['system', 'user', 'bot'],
        num_rows: 6
    })
})

## Предобработка датасета

In [10]:
print(tokenizer.bos_token_id)
print(tokenizer.eos_token_id)
print(tokenizer.decode([1]))
print(tokenizer.decode([2]))

1
2
<s>
</s>


In [11]:
print(tokenizer('system', add_special_tokens=False))
print(tokenizer('user', add_special_tokens=False))
print(tokenizer('bot', add_special_tokens=False))

{'input_ids': [1788], 'attention_mask': [1]}
{'input_ids': [1404], 'attention_mask': [1]}
{'input_ids': [9225], 'attention_mask': [1]}


In [12]:
tokenizer('<s>system привет как дела</s><s>',truncation=True,
        max_length=3584,
        padding=False,
        return_tensors=None)

{'input_ids': [1, 1, 1788, 1695, 7616, 5413, 24280, 2, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [13]:
#  promt = f"""<s>system
# {data_point['system']}</s><s>user
# {data_point['user']}</s><s>bot
# {data_point['bot']}</s>
#     """
    #     print(promt)

In [14]:
CUTOFF_LEN = 3584

SYSTEM = tokenizer('system', add_special_tokens=False)['input_ids'][0]
USER = tokenizer('user', add_special_tokens=False)['input_ids'][0]
BOT = tokenizer('bot', add_special_tokens=False)['input_ids'][0]
print(f'system: {SYSTEM}')
print(f'user: {USER}')
print(f'bot: {BOT}')

system: 1788
user: 1404
bot: 9225


In [15]:

# def generate_prompt(data_point):
#     tokenizer_full_promt = {'input_ids': [], 'attention_mask': [], 'labels': []}
#     for key in data_point.keys():
#         tokenizer_promt = tokenize(data_point[key])
#         if key == 'system':
#             tokenizer_full_promt['input_ids'].append(1)
#             tokenizer_full_promt['input_ids'].append(SYSTEM)
#             tokenizer_full_promt['labels'].append(1)
#             tokenizer_full_promt['labels'].append(SYSTEM)
#             tokenizer_full_promt['attention_mask'] += [1, 1]
#         elif key == 'user':
#             tokenizer_full_promt['input_ids'].append(1)
#             tokenizer_full_promt['input_ids'].append(USER)
#             tokenizer_full_promt['labels'].append(1)
#             tokenizer_full_promt['labels'].append(USER)
#             tokenizer_full_promt['attention_mask'] += [1, 1]
    
#         elif key == 'bot':
#             tokenizer_full_promt['input_ids'].append(1)
#             tokenizer_full_promt['input_ids'].append(BOT)
#             tokenizer_full_promt['labels'].append(1)
#             tokenizer_full_promt['labels'].append(BOT)
#             tokenizer_full_promt['attention_mask'] += [1, 1]
            
#         else:
#             print('Что то пошло не так')
            
#         for key_promt in tokenizer_promt.keys():
#             tokenizer_full_promt[key_promt] += tokenizer_promt[key_promt]
        
#         tokenizer_full_promt['input_ids'].append(2)
#         tokenizer_full_promt['labels'].append(2)
#         tokenizer_full_promt['attention_mask'].append(1)
        
        
#     return tokenizer_full_promt
 
    
# def tokenize (prompt, add_eos_token=False):
#     result = tokenizer(
#         prompt,
#         truncation=True,
#         max_length=CUTOFF_LEN,
#         padding=False,
#         return_tensors=None,
#         add_special_tokens=False
#     )
#     if (
#         result["input_ids"][-1] != tokenizer.eos_token_id and len(result["input_ids"]) < CUTOFF_LEN
#         and add_eos_token
#     ):
        
#         result["input_ids"].append(tokenizer.eos_token_id)
#         result["attention_mask"].append(1)
        
        
    
#     result["labels"] = result["input_ids"].copy()

#     return result
 
# def generate_and_tokenize_prompt(data_point):
#     tokenized_full_prompt = generate_prompt(data_point)
#     return tokenized_full_prompt

In [16]:
CUTOFF_LEN = 3584

def generate_prompt(data_point):
    promt = f"""<s>system
{data_point['system']}</s><s>user
{data_point['user']}</s><s>bot
{data_point['bot']}</s>"""
    #     print(promt)
    return promt
 
    
def tokenize (prompt, add_eos_token=True):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=CUTOFF_LEN,
        padding=False,
        return_tensors=None,
    )
    if (
        result["input_ids"][-1] != tokenizer.eos_token_id and len(result["input_ids"]) < CUTOFF_LEN
        and add_eos_token
    ):
        
        result["input_ids"].append(tokenizer.eos_token_id)
        result["attention_mask"].append(1)
        
        
    
    result["labels"] = result["input_ids"].copy()

    return result
 
def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)
    tokenized_full_prompt = tokenize(full_prompt)
#     print(tokenized_full_prompt)
    return tokenized_full_prompt

In [17]:
print(generate_and_tokenize_prompt(data["train"][0]))

{'input_ids': [1, 1, 1788, 13, 30041, 29982, 1866, 29942, 753, 642, 29919, 25366, 10539, 989, 933, 606, 20658, 6052, 1866, 23567, 1229, 2, 1, 1404, 13, 30054, 29972, 29942, 753, 717, 1866, 11650, 1868, 23567, 1229, 6495, 20658, 6052, 606, 10539, 989, 933, 29889, 14966, 8160, 11213, 4054, 23761, 490, 21986, 27768, 2494, 16919, 10786, 531, 1186, 2583, 1282, 989, 1840, 606, 5023, 13, 16396, 29988, 1155, 3327, 785, 6408, 662, 587, 11752, 29970, 665, 1578, 693, 3162, 29871, 8089, 25656, 13781, 29892, 490, 20935, 11883, 10689, 576, 642, 733, 29904, 1093, 4715, 570, 16186, 1779, 4394, 16844, 29988, 18992, 3029, 5406, 3325, 3541, 665, 1186, 13174, 1282, 14820, 1447, 11106, 29889, 2081, 2194, 8847, 3378, 1257, 8440, 730, 1892, 17683, 26907, 30002, 6052, 606, 8433, 507, 4962, 1587, 6785, 5752, 29889, 2, 1, 9225, 13, 29961, 10998, 8489, 22099, 30074, 29910, 29988, 1155, 3327, 742, 525, 16553, 2396, 525, 29957, 587, 11752, 29970, 665, 1578, 693, 3162, 29871, 8089, 25656, 13781, 29892, 490, 20935, 

In [18]:
train_data = (
    data["train"].map(generate_and_tokenize_prompt)
)

val_data = (
    data["validation"].map(generate_and_tokenize_prompt)
)

  0%|          | 0/22 [00:00<?, ?ex/s]

  0%|          | 0/6 [00:00<?, ?ex/s]

## Обучаем модель

In [19]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [20]:
BATCH_SIZE = 4
MICRO_BATCH_SIZE = 2
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE
LEARNING_RATE = 3e-4
TRAIN_STEPS = 100
OUTPUT_DIR = "/kaggle/working/tmp"

training_arguments = transformers.TrainingArguments(
            per_device_train_batch_size=MICRO_BATCH_SIZE,
            gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
#             warmup_steps=200,
            max_steps=TRAIN_STEPS,
            learning_rate=LEARNING_RATE,
            fp16=True,
            logging_steps=10,
            optim="adamw_torch",
            evaluation_strategy="steps",
            save_strategy="steps",
            eval_steps=10,
            save_steps=10,
            output_dir=OUTPUT_DIR,
            save_total_limit=10,
            load_best_model_at_end=True,
            report_to=None,
            overwrite_output_dir=True, # Overwrite the content of the output dir
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [21]:
data_collator = transformers.DataCollatorForSeq2Seq(
    tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
)

In [22]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    args=training_arguments,
    data_collator=data_collator
)
model = torch.compile(model)
trainer.train()
model.save_pretrained(OUTPUT_DIR)

Step,Training Loss,Validation Loss
10,1.139700,nan
20,0.716100,0.594529
30,0.271100,0.553778
40,0.110700,0.521688
50,0.057300,0.536120
60,0.040300,0.554408
70,0.033600,0.561207
80,0.032200,0.574507
90,0.031200,0.575170
100,0.030100,0.576941


## Конвертирование модели в gguf

In [ ]:
import os
os._exit(00) # Перезапустить ядро

In [1]:
%mkdir /kaggle/tmp
%cd /kaggle/tmp
%ls

/kaggle/tmp


In [2]:
self_instruct_dir = '/kaggle/working/rulm/self_instruct'

checkpoint = "/kaggle/working/tmp/checkpoint-40"

merged_model_name = '/kaggle/tmp/merged_test_model.pt'

self_instruct_dir

'/kaggle/working/rulm/self_instruct'

In [3]:
%cd {self_instruct_dir}

/kaggle/working/rulm/self_instruct


### Склеиваем вместе обученные адаптеры с базовой моделью, сохраняем результат в формат PyTorch
##### Параметры
1. чекпоинт лучшей эпохи
2. куда сохраняем
3. Используем cpu или cuda
4. Выгружаем ли половину тензеров в оперативку

In [4]:
!python -m src.tools.convert_to_native {checkpoint} {merged_model_name} --device=cuda --enable_offloading
# assert (output_dir / merged_model_name).exists()

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
100%|█████████████████████████████████████████| 291/291 [00:05<00:00, 52.85it/s]
Saving state_dict...


In [5]:
%cd /kaggle/tmp/
%ls

/kaggle/tmp
merged_test_model.pt


### Конвертируем склеенную модель в 16-битный формат GGUF (llama.cpp)

In [ ]:
import os
os._exit(00) # Перезапустить ядро

In [7]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("IlyaGusev/saiga2_7b_lora", use_fast=False)
tokenizer.save_pretrained('/kaggle/working/tmp/checkpoint-40/')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


('/kaggle/working/tmp/checkpoint-40/tokenizer_config.json',
 '/kaggle/working/tmp/checkpoint-40/special_tokens_map.json',
 '/kaggle/working/tmp/checkpoint-40/tokenizer.model',
 '/kaggle/working/tmp/checkpoint-40/added_tokens.json')

In [8]:
%cd /kaggle/working/llama.cpp

/kaggle/working/llama.cpp


In [9]:
%%capture
!python3 -m pip install -r requirements.txt

In [10]:
model_dir = '/kaggle/tmp/merged_test_model.pt'
checkpoint = "/kaggle/working/tmp/checkpoint-40"
output_model = "/kaggle/tmp/model-f16.gguf"

In [11]:
!python convert.py {model_dir} --vocab-dir {checkpoint} --outfile {output_model} --outtype f16 --ctx 4096

Loading model file /kaggle/tmp/merged_test_model.pt
params = Params(n_vocab=32000, n_embd=4096, n_layer=32, n_ctx=4096, n_ff=11008, n_head=32, n_head_kv=32, f_norm_eps=1e-05, rope_scaling_type=None, f_rope_freq_base=None, f_rope_scale=None, n_orig_ctx=None, rope_finetuned=None, ftype=<GGMLFileType.MostlyF16: 1>, path_model=PosixPath('/kaggle/tmp'))
Loading vocab file '/kaggle/working/tmp/checkpoint-40/tokenizer.model', type 'spm'
tok_embeddings.weight                            -> token_embd.weight                        | F16    | [32000, 4096]
layers.0.attention.wq.weight                     -> blk.0.attn_q.weight                      | F16    | [4096, 4096]
layers.0.attention.wk.weight                     -> blk.0.attn_k.weight                      | F16    | [4096, 4096]
layers.0.attention.wv.weight                     -> blk.0.attn_v.weight                      | F16    | [4096, 4096]
layers.0.attention.wo.weight                     -> blk.0.attn_output.weight                 | F1

In [12]:
%cd /kaggle/tmp/
%ls

/kaggle/tmp
merged_test_model.pt  model-f16.gguf


### Квантуем моедль в 4 бита и 8 бит

In [ ]:
import os
os._exit(00) # Перезапустить ядро

In [1]:
%cd /kaggle/working/llama.cpp

/kaggle/working/llama.cpp


In [2]:
!make quantize

I llama.cpp build info: 
I UNAME_S:   Linux
I UNAME_P:   x86_64
I UNAME_M:   x86_64
I CFLAGS:    -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG  -std=c11   -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wshadow -Wstrict-prototypes -Wpointer-arith -Wmissing-prototypes -Werror=implicit-int -Werror=implicit-function-declaration -Wdouble-promotion -pthread -march=native -mtune=native 
I CXXFLAGS:  -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG  -std=c++11 -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wmissing-declarations -Wmissing-noreturn -pthread  -Wno-array-bounds -Wno-format-truncation -Wextra-semi -march=native -mtune=native 
I NVCCFLAGS:  -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG  -std=c++11 -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wmissing-declarations -Wmissing-noreturn -pthread    -Wno-pedantic -Xcompiler "-Wno-array-bounds -Wno-format-truncation -Wextra-semi -march=native -mt

In [3]:
model_gguf = "/kaggle/tmp/model-f16.gguf"
quant_model = "/kaggle/working/model-q4_0.gguf"
quantization_type = "q4_0" #@param ["q4_0", "q4_1"] {allow-input: true}

In [4]:
! ./quantize {model_gguf} {quant_model} {quantization_type}

main: build = 1574 (8406b09)
main: built with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu
main: quantizing '/kaggle/tmp/model-f16.gguf' to '/kaggle/working/model-q4_0.gguf' as Q4_0
llama_model_loader: loaded meta data with 15 key-value pairs and 291 tensors from /kaggle/tmp/model-f16.gguf (version GGUF V3 (latest))
llama_model_loader: - tensor    0:                token_embd.weight f16      [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:              blk.0.attn_q.weight f16      [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    2:              blk.0.attn_k.weight f16      [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_v.weight f16      [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    4:         blk.0.attn_output.weight f16      [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_gate.weight f16      [  4096, 11008,     1,     1 ]
llama_model_loade

## Проверка gguf модели

In [5]:
%cd ~
!git clone --recursive https://github.com/ggerganov/llama.cpp.git
%cd llama.cpp
!make LLAMA_CUBLAS=1 -j libllama.so

# HACK: Use custom compiled libllama.so
%cp ~/llama.cpp/libllama.so /opt/conda/lib/python3.10/site-packages/llama_cpp/libllama.so

/root
Cloning into 'llama.cpp'...
remote: Enumerating objects: 12546, done.
remote: Counting objects: 100% (5693/5693), done.
remote: Compressing objects: 100% (472/472), done.
remote: Total 12546 (delta 5487), reused 5231 (delta 5221), pack-reused 6853
Receiving objects: 100% (12546/12546), 14.92 MiB | 2.01 MiB/s, done.
Resolving deltas: 100% (8755/8755), done.
/root/llama.cpp
I llama.cpp build info: 
I UNAME_S:   Linux
I UNAME_P:   x86_64
I UNAME_M:   x86_64
I CFLAGS:    -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_USE_CUBLAS -I/usr/local/cuda/include -I/opt/cuda/include -I/targets/x86_64-linux/include  -std=c11   -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wshadow -Wstrict-prototypes -Wpointer-arith -Wmissing-prototypes -Werror=implicit-int -Werror=implicit-function-declaration -Wdouble-promotion -pthread -march=native -mtune=native 
I CXXFLAGS:  -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_USE_CUBLAS -I/usr/local/cuda/inc

In [6]:
# import fire
from llama_cpp import Llama
from tqdm import tqdm

In [11]:
SYSTEM_PROMPT = "Ты извлекаешь термины и определения из текста"
SYSTEM_TOKEN = 1788
USER_TOKEN = 1404
BOT_TOKEN = 9225
LINEBREAK_TOKEN = 13

top_k=40
top_p=0.5
temperature=0.01
repeat_penalty=1.1


ROLE_TOKENS = {
    "user": USER_TOKEN,
    "bot": BOT_TOKEN,
    "system": SYSTEM_TOKEN
}


def get_message_tokens(model, role, content):
    message_tokens = model.tokenize(content.encode("utf-8"))
    message_tokens.insert(1, ROLE_TOKENS[role])
    message_tokens.insert(2, LINEBREAK_TOKEN)
    message_tokens.append(model.token_eos())
    return message_tokens


def get_system_tokens(model):
    system_message = {
        "role": "system",
        "content": SYSTEM_PROMPT
    }
    return get_message_tokens(model, **system_message)

def chat_saiga(message, model):
    system_tokens = get_system_tokens(model)
    tokens = system_tokens
    # model.eval(tokens)
    
    message_tokens = get_message_tokens(model=model, role="user", content=message)
    role_tokens = [model.token_bos(), BOT_TOKEN, LINEBREAK_TOKEN]
    tokens += message_tokens + role_tokens
    # print(tokens)
    # detokenize = model.detokenize(tokens)
    # print(model.tokenize(full_prompt))
    generator = model.generate(
        tokens,
        top_k = top_k,
        top_p = top_p,
        temp = temperature,
        repeat_penalty = repeat_penalty,
        reset = True
    )
    # print(len([token for token in generator]))
    
    result_list = []
    
    for token in generator:
        token_str = model.detokenize([token]).decode("utf-8", errors="ignore")
        tokens.append(token)
        if token == model.token_eos():
            break
        print(token_str, end="", flush=True)
        result_list.append(token_str)
    return ''.join(result_list)

In [8]:
%cd /kaggle/tmp
%ls

/kaggle/tmp
merged_test_model.pt  model-f16.gguf


In [9]:
%%capture
try:
    del model
except:
    pass

# model_path = '/kaggle/working/model-q4_0.gguf'
model_path = '/kaggle/working/model-q4_0.gguf'
n_ctx = 3096 #

model = Llama(
        model_path = model_path,
        n_ctx = n_ctx,
        n_gpu_layers=-1
) # n_gpu_layers=-1  параметр для переноса модели на GPU

ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 1 CUDA devices:
  Device 0: Tesla P100-PCIE-16GB, compute capability 6.0
llama_model_loader: loaded meta data with 16 key-value pairs and 291 tensors from /kaggle/working/model-q4_0.gguf (version GGUF V3 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q4_0     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:              blk.0.attn_q.weight q4_0     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    2:              blk.0.attn_k.weight q4_0     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_v.weight q4_0     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    4:         blk.0.attn_output.weight q4_0     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_gate.weight q4_0     [  4096, 11008,     1,     1 ]
llama_model_loader: - ten

In [14]:
%%time
promt = """
Извлеки из данного текста все определения и термины.
Вывод должен быть в виде списка словарей с ключами term и definition

Машинное обучение (ML) — это использование математических моделей данных, которые помогают компьютеру обучаться без непосредственных инструкций.
Оно считается одной из форм искусственного интеллекта (ИИ).
"""
chat_saiga(promt, model)

[

Llama.generate: prefix-match hit


{'term':'Машинное обучение (ML)', 'definition': 'использование математических моделей данных, которые помогают компьютеру обучаться без непосредственных инструкций.'}]CPU times: user 2.86 s, sys: 47.7 ms, total: 2.91 s
Wall time: 2.87 s


"[{'term':'Машинное обучение (ML)', 'definition': 'использование математических моделей данных, которые помогают компьютеру обучаться без непосредственных инструкций.'}]"

## Проверяем модель на адекватность

In [ ]:
DEFAULT_MESSAGE_TEMPLATE = "<s>{role}\n{content}</s>"
DEFAULT_RESPONSE_TEMPLATE = "<s>bot\n"
DEFAULT_SYSTEM_PROMPT = "Ты извлекаешь термины и определения из текста"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class Conversation:
    def __init__(
        self,
        message_template = DEFAULT_MESSAGE_TEMPLATE,
        system_prompt = DEFAULT_SYSTEM_PROMPT,
        response_template = DEFAULT_RESPONSE_TEMPLATE
    ):
        self.message_template = message_template
        self.response_template = response_template
        self.messages = [{
            "role": "system",
            "content": system_prompt
        }]

    def add_user_message(self, message):
        self.messages.append({
            "role": "user",
            "content": message
        })

    def add_bot_message(self, message):
        self.messages.append({
            "role": "bot",
            "content": message
        })

    def get_prompt(self, tokenizer):
        final_text = ""
        for message in self.messages:
            message_text = self.message_template.format(**message)
            final_text += message_text
        final_text += DEFAULT_RESPONSE_TEMPLATE
        return final_text.strip()



def generate(model, tokenizer, prompt, generation_config):
    data = tokenizer(prompt,
                     return_tensors="pt",
                     add_special_tokens=False,
#                      padding=True,
#                     truncation=True
                    )
    data = {k: v.to(device) for k, v in data.items()}
    
    output_ids = model.generate(
        **data,
        generation_config = generation_config,
#         remove_invalid_values = True
    )[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    return output.strip()

In [ ]:
%%time
def get_message(inputs):
    result = []
    for inp in inputs:
        conversation = Conversation()
        conversation.add_user_message(inp)
        prompt = conversation.get_prompt(tokenizer)
        print(prompt)
        output = generate(model, tokenizer, prompt, generation_config)
        result.append(output)
    return result

In [ ]:
p = """
Извлеки из данного текста все определения и термины.
Вывод должен быть в виде списка словарей с ключами term и definition

Линейная модель — модель, отображающая состояние или функционирование системы таким образом, что все взаимозависимости в ней принимаются линейными (см. Линейная зависимость, Линейность в экономике). Соответственно, она может формулироваться в виде одного линейного уравнения или системы линейных уравнений.
"""
get_message([p])

In [ ]:
from huggingface_hub import notebook_login
 
notebook_login()

In [ ]:
model.push_to_hub("YarKo69/Saiga_2_7b_fine_tune_custom_data", use_auth_token=True)

In [ ]:
{"system": "Ты извлекаешь термины и определения из текста", "user": "Извлеки из данного текста все определения и термины. Вывод должен быть в виде списка словарей с ключами term и definition\n Суффикс - это значимая часть слова, которая находится после корня и перед окончанием, служит для образования новых слов.", "bot": "[{'term':'Суффикс', 'definition': 'значимая часть слова, которая находится после корня и перед окончанием, служит для образования новых слов.'}]"}


## Гайды по обучению
* [Омега полезный](https://www.mlexpert.io/machine-learning/tutorials/alpaca-fine-tuning)
* [Тоже полезный](https://brev.dev/blog/fine-tuning-llama-2-your-own-data#1-load-dataset)